In [21]:
from tkinter import *
from tkinter import filedialog, messagebox
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from ttkthemes import ThemedStyle


class App:
    def __init__(self):
        self.root = Tk()
        self.root['bg'] = 'black'
        self.root.title("Modelo de Machine Learning")
        self.root.geometry("400x400")
        self.root.resizable(False, False)
        # self.root.eval('tk::PlaceWindow . center')
        # Obtener las dimensiones de la pantalla
        screen_width = self.root.winfo_screenwidth()
        screen_height = self.root.winfo_screenheight()

        # Calcular las coordenadas para centrar la ventana
        x = (screen_width - self.root.winfo_reqwidth()) // 2
        y = (screen_height - 200 - self.root.winfo_reqheight()) // 2

        # Establecer las coordenadas para centrar la ventana
        self.root.geometry(f"+{x}+{y}")
        self.root.protocol("WM_DELETE_WINDOW", self.root.destroy)
        label_title = Label(self.root, text="Proyecto de Algoritmica Avanzada", font=("Arial", 14, "bold"))
        label_title.pack(pady=10)
        style = ThemedStyle(self.root)
        style.set_theme("default")
        # Mostramos mensaje para ayudar al usuario al momento de querer seleccionar un nuevo modelo de aprendizaje
        messagebox.showinfo("Ayuda", "Para seleccionar un nuevo modelo de aprendizaje, primero debe cargar un archivo CSV.", parent=self.root)
        # Inicializacion de variables
        self.df = None
        self.columnas_seleccionadas = []
        self.columna_objetivo = None
        self.modelo_linear = None
        self.modelo_tree = None
        self.modelo_kneighbors = None
        self.resultado_linear = None
        self.resultado_tree = None
        self.resultado_kneighbors = None
        self.columnas_variables = []

        # Boton Cargar Archivo que llama a la funcion cargar_archivo
        self.button_cargar_archivo = Button(self.root, text="Cargar archivo", command=self.cargar_archivo)
        self.button_cargar_archivo.pack(pady=10)
        # Boton Seleccionar Modelo que llama a la funcion seleccionar_modelo
        self.button_seleccionar_modelo = Button(self.root, text="Seleccionar modelo",
        command=self.seleccionar_modelo)
        self.button_seleccionar_modelo.pack(pady=10)
        # Boton Seleccionar Columnas que llama a la funcion seleccionar_columnas
        self.button_seleccionar_columnas = Button(self.root, text="Seleccionar columnas",
        command=self.seleccionar_columnas)
        self.button_seleccionar_columnas.pack(pady=10)
        # Boton Seleccionar Columna Objetivo que llama a la funcion seleccionar_columna_objetivo
        self.button_entrenar_modelo = Button(self.root, text="Seleccionar columna objetivo", command=self.seleccionar_columna_objetivo)
        self.button_entrenar_modelo.pack(pady=10)
        # Boton Score Linear Regression que llama a la funcion mostrar_score_linear
        self.button_score_linear= Button(self.root, text="Score de Linear Regression",
        command=self.mostrar_score_linear)
        self.button_score_linear.pack(pady=10)
        # Boton Score de Tree que llama a la funcion mostrar_score_tree
        self.button_score_tree = Button(self.root, text="Score de tree",
        command=self.mostrar_score_tree)
        self.button_score_tree.pack(pady=10)
        # Boton Score de KNeighbors que llama a la funcion mostrar_score_kneighbors
        self.button_score_kneighbors = Button(self.root, text="Score de KNeighbors",
        command=self.mostrar_score_kneighbors)
        self.button_score_kneighbors.pack(pady=10)

        # Inicia la ventana principal y la mantiene en ejecucion
        self.root.mainloop()

    # Funcion que permite cargar un archivo CSV y llama a la funcion limpiar_datos
    def cargar_archivo(self):
        archivo = filedialog.askopenfilename(filetypes=(("Archivos CSV", "*.csv"),))
        if archivo:
            try:
                # Los datos del .csv se guardan en df
                self.df = pd.read_csv(archivo)
                self.limpiar_datos()
                messagebox.showinfo("Éxito", "Archivo cargado correctamente.", parent=self.root)
            except Exception as e:
                messagebox.showerror("Error", str(e))
    # Funcion que permite eliminar columnas no deseadas, rellena con 0 los valores nulo y se llama en la funcion cargar_archivo
    def limpiar_datos(self):
        columnas_eliminar = []
        self.columnas_seleccionadas = []
        self.columna_objetivo = None
        self.modelo_linear = None
        self.modelo_tree = None
        self.modelo_kneighbors = None
        self.resultado_linear = None
        self.resultado_tree = None
        self.resultado_kneighbors = None
        for columna in self.df.columns:
            print(f"La columna {columna} es de tipo {self.df[columna].dtype}")
            # Si la columna es numerica y tiene ids, se elimina
            if self.df[columna].dtype == "int64":
                if self.df[columna].min() == 1 and self.df[columna].max() == len(self.df):
                    print(f"La columna {columna} es de tipo {self.df[columna].dtype} y tiene ids.")
                    columnas_eliminar.append(columna)
            if self.df[columna].dtype == "object":
                    if len(self.df[columna].unique()) > 10:
                        print(f"La columna {columna} es de tipo object y tiene más de 10 valores únicos.")
                        columnas_eliminar.append(columna)
                    else:
                        # Convierte las columnas categorias a numeros
                        self.df[columna] = pd.Categorical(self.df[columna])
                        self.df[columna] = self.df[columna].cat.codes
                        self.df[columna] = self.df[columna] - self.df[columna].min()
                        print(f"Valores únicos de la columna {columna}: {self.df[columna].unique()}")
                        pass
            # Verifica si la columna contienen valores nulos y rellena con el promedio
            if self.df[columna].isnull().values.any() and columna not in columnas_eliminar:
                print(f"La columna {columna} contiene valores nulos.")
                self.df[columna] = self.df[columna].fillna(self.df[columna].mean())
        print(f"Eliminando columnas {columnas_eliminar}")
        self.df = self.df.drop(columns=columnas_eliminar)

        # self.df = self.df.fillna(0)

    # Funcion que permite seleccionar las columnas que se van a utilizar para entrenar el modelo
    def seleccionar_columnas(self):
        # Crea una ventana para seleccionar las columnas
        ventana_columnas = Toplevel(self.root)
        ventana_columnas.title("Seleccionar columnas")
        ventana_columnas.geometry("400x400")
        ventana_columnas.resizable(False, False)
        # Obtener las dimensiones de la pantalla
        screen_width = ventana_columnas.winfo_screenwidth()
        screen_height = ventana_columnas.winfo_screenheight()

        # Calcular las coordenadas para centrar la ventana
        x = (screen_width - ventana_columnas.winfo_reqwidth()) // 2
        y = (screen_height - 40 - ventana_columnas.winfo_reqheight()) // 2

        # Establecer las coordenadas para centrar la ventana
        ventana_columnas.geometry(f"+{x}+{y}")
        frame_columnas = Frame(ventana_columnas)
        frame_columnas.pack()

        label_columnas = Label(frame_columnas, text="Columnas disponibles:")
        label_columnas.pack()

        scrollbar_columnas = Scrollbar(frame_columnas, orient="vertical")
        listbox_columnas = Listbox(frame_columnas, selectmode="extended", yscrollcommand=scrollbar_columnas.set)

        for columna in self.df.columns:
            listbox_columnas.insert(END, columna)

        for i, columna in enumerate(self.df.columns):
            var = IntVar(value=0)
            self.columnas_variables.append(var)
            checkbox = Checkbutton(frame_columnas, text=columna, variable=var)
            checkbox.pack(anchor="w")

        listbox_columnas.pack(side="left", fill="y")
        scrollbar_columnas.pack(side="right", fill="y")
        # Guarda las columnas seleccionadas en una lista
        def seleccionar():
            seleccion = listbox_columnas.curselection()
            self.columnas_seleccionadas = list(seleccion)

            ventana_columnas.destroy()
            messagebox.showinfo("Éxito", "Columnas seleccionadas correctamente.")

        button_seleccionar = Button(ventana_columnas, text="Seleccionar", command=seleccionar)
        button_seleccionar.pack()

    # Funcion que permite seleccionar el modelo que se va a utilizar para entrenar
    def seleccionar_modelo(self):
        ventana_modelo = Toplevel(self.root)
        ventana_modelo.title("Seleccionar Modelo:")
        ventana_modelo.geometry("400x400")
        ventana_modelo.resizable(False, False)
        # Obtener las dimensiones de la pantalla
        screen_width = ventana_modelo.winfo_screenwidth()
        screen_height = ventana_modelo.winfo_screenheight()

        # Calcular las coordenadas para centrar la ventana
        x = (screen_width - ventana_modelo.winfo_reqwidth()) // 2
        y = (screen_height - 40 - ventana_modelo.winfo_reqheight()) // 2

        # Establecer las coordenadas para centrar la ventana
        ventana_modelo.geometry(f"+{x}+{y}")
        # ventana_modelo.eval('tk::PlaceWindow . center')

        label_modelo = Label(ventana_modelo, text="Seleccione el tipo de modelo:")
        label_modelo.pack(pady=10)

        seleccion_modelo = StringVar()
        seleccion_modelo.set(None)
        radio_button_linear= Radiobutton(ventana_modelo, text="linear", variable=seleccion_modelo, value="linear")
        radio_button_linear.pack()

        radio_button_tree = Radiobutton(ventana_modelo, text="tree", variable=seleccion_modelo, value="tree")
        radio_button_tree.pack()

        radio_button_kneighbors = Radiobutton(ventana_modelo, text="kneighbors", variable=seleccion_modelo, value="kneighbors")
        radio_button_kneighbors.pack()

        # Guarda el modelo seleccionado en una variable
        def seleccionar():
            modelo = seleccion_modelo.get()
            if modelo == "linear":
                self.modelo_linear = LinearRegression()
            elif modelo == "tree":
                self.modelo_tree = DecisionTreeClassifier()
            elif modelo == "kneighbors":
                self.modelo_kneighbors = KNeighborsClassifier()
            ventana_modelo.destroy()
            messagebox.showinfo("Éxito", f"Modelo {modelo} seleccionado correctamente.")

        button_seleccionar = Button(ventana_modelo, text="Seleccionar", command=seleccionar)
        button_seleccionar.pack(pady=10)
    # Funcion que permite seleccionar una columna objetivo y se llama cuando se selecciona la columna objetivo
    def seleccionar_columna_objetivo(self):
        if self.df is None or self.df.empty:
            messagebox.showerror("Error", "Cargue un archivo CSV antes de entrenar el modelo.")
            return

        if not self.columnas_seleccionadas:
            messagebox.showerror("Error", "Seleccione al menos una columna de entrenamiento.")
            return

        if self.modelo_linear is None and self.modelo_tree is None and self.modelo_kneighbors is None:
            messagebox.showerror("Error", "Seleccione un modelo antes de entrenar.")
            return

        ventana_objetivo = Toplevel(self.root)
        ventana_objetivo.title("Seleccionar columna objetivo:")
        ventana_objetivo.geometry("400x400")
        # Obtener las dimensiones de la pantalla
        screen_width = ventana_objetivo.winfo_screenwidth()
        screen_height = ventana_objetivo.winfo_screenheight()

        # Calcular las coordenadas para centrar la ventana
        x = (screen_width - ventana_objetivo.winfo_reqwidth()) // 2
        y = (screen_height - 40 - ventana_objetivo.winfo_reqheight()) // 2

        # Establecer las coordenadas para centrar la ventana
        ventana_objetivo.geometry(f"+{x}+{y}")
        frame_objetivo = Frame(ventana_objetivo)
        frame_objetivo.pack(pady=10)

        label_objetivo = Label(frame_objetivo, text="Columna objetivo:")
        label_objetivo.pack()

        scrollbar_objetivo = Scrollbar(frame_objetivo, orient="vertical")
        listbox_objetivo = Listbox(frame_objetivo, selectmode="single", yscrollcommand=scrollbar_objetivo.set)

        for columna in self.df.columns:
            listbox_objetivo.insert(END, columna)

        listbox_objetivo.pack(side="left", fill="y")
        scrollbar_objetivo.pack(side="right", fill="y")

        # Guarda la columna objetivo seleccionada en una variable
        def seleccionar_objetivo():
            seleccion = listbox_objetivo.curselection()
            if seleccion:
                self.columna_objetivo = seleccion[0]
                ventana_objetivo.destroy()
                self.entrenar_modelo_ml()
                messagebox.showinfo("Éxito", "Modelo entrenado correctamente.")
            else:
                messagebox.showerror("Error", "Seleccione una columna objetivo.")

        button_seleccionar_objetivo = Button(ventana_objetivo, text="Seleccionar", command=seleccionar_objetivo)
        button_seleccionar_objetivo.pack()

    # Funcion que permite entrenar el modelo seleccionado
    def entrenar_modelo_ml(self):
        x = self.df.iloc[:, [i for i, var in enumerate(self.columnas_variables) if var.get() == 1]]
        y = self.df.iloc[:, self.columna_objetivo]
        train_x, test_x, train_y, test_y = train_test_split(x, y)
        print(f'Esto es X: ' + str(x)) 
        print(f'Esto es Y: ' + str(y))

        if self.modelo_linear:
            self.modelo_linear.fit(train_x, train_y)
            self.resultado_linear = self.modelo_linear.score(test_x, test_y) * 100.0
        elif self.modelo_tree:
            self.modelo_tree.fit(train_x, train_y)
            self.resultado_tree = self.modelo_tree.score(test_x, test_y) * 100.0
        elif self.modelo_kneighbors:
            self.modelo_kneighbors.fit(train_x, train_y)
            self.resultado_kneighbors = self.modelo_kneighbors.score(test_x, test_y) * 100.0
            

    def mostrar_score_linear(self):
        if self.modelo_linear:
            messagebox.showinfo("Score de linear", f"Score (linear): {round(self.resultado_linear, 4)}%")
        else:
            messagebox.showerror("Error", "Entrene el modelo de regresión antes de mostrar el Score.")

    def mostrar_score_tree(self):
        if self.modelo_tree:
            messagebox.showinfo("Score de tree", f"Score (tree): {round(self.resultado_tree, 4)}%")
        else:
            messagebox.showerror("Error", "Entrene el modelo de tree antes de mostrar el Score.")

    def mostrar_score_kneighbors(self):
        if self.modelo_kneighbors:
            messagebox.showinfo("Score de kneighbors", f"Score (kneighbors): {round(self.resultado_kneighbors, 4)}%")
        else:
            messagebox.showerror("Error", "Entrene el modelo de kneighbors antes de mostrar el Score.")


app = App()


La columna PassengerId es de tipo int64
La columna PassengerId es de tipo int64 y tiene ids.
La columna Survived es de tipo int64
La columna Pclass es de tipo int64
La columna Name es de tipo object
La columna Name es de tipo object y tiene más de 10 valores únicos.
La columna Sex es de tipo object
Valores únicos de la columna Sex: [1 0]
La columna Age es de tipo float64
La columna Age contiene valores nulos.
La columna SibSp es de tipo int64
La columna Parch es de tipo int64
La columna Ticket es de tipo object
La columna Ticket es de tipo object y tiene más de 10 valores únicos.
La columna Fare es de tipo float64
La columna Cabin es de tipo object
La columna Cabin es de tipo object y tiene más de 10 valores únicos.
La columna Embarked es de tipo object
Valores únicos de la columna Embarked: [3 1 2 0]
Eliminando columnas ['PassengerId', 'Name', 'Ticket', 'Cabin']
